# Programming Exercises

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
df_size = len(df)
disasters_size = len(df_train[df_train.target == 1])
print(f"Training set size: {df_size} (Distaters: {disasters_size}, Other: {df_size-disasters_size})")

df_test_size = len(df_test)
print(f"Testing set size: {df_test_size}")

Training set size: 7613 (Distaters: 3271, Other: 4342)
Testing set size: 3263


In [4]:
df.location[df.location.notna()].iloc[108]

'italy'

## 1
### a
7613 Training Points (3271 Disasters), 3263 Testing Points

In [6]:
# ID is uselesss and Location may bee to noisy
df_train = d_train.drop(columns=["id", "location"])

In [9]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words as englishwords
from nltk.stem import WordNetLemmatizer
def tokenize(string, lower=True, punctuation=True, stops=True, lemmatize=True, check_english=True):
    stop_words = set(stopwords.words("english"))
    english_words = set(englishwords.words())
    words = word_tokenize(string)
    lzr = WordNetLemmatizer() 

    if lower:
        words = [w.lower() for w in words]
    if punctuation:
        words = [w for w in words if w.isalnum()]
    if stops:
        words = [w for w in words if w not in stop_words]
    if lemmatize:
        words = [lzr.lemmatize(w) for w in words]
    # Takes long to run, use with caution
    if check_english:
        words = [w for w in words if w in english_words]
    return words

In [10]:
from nltk.stem import WordNetLemmatizer
lzr = WordNetLemmatizer()
df_train.text = df.text.apply(lambda x: tokenize(x))
df_train.keywords = df.keyword.apply(lambda x: x.lower() if isinstance(x, str) else x)

In [1]:
df_train.to_csv("new_train.csv")

NameError: name 'df' is not defined

In [94]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv("filtered_train.csv", index_col=False)
df = df.rename(columns={"target": "is_disaster"})

In [95]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True, min_df=20)
sample = df.text.iloc[0]
X = vectorizer.fit_transform(df.text.values)
bag_of_words = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
df = pd.concat([df.drop(columns="Unnamed: 0"), bag_of_words], axis=1, join="inner")

In [96]:
X_train, X_dev, y_train, y_dev = train_test_split(df.drop(columns=["is_disaster", "keyword", "text"]), df.is_disaster, train_size=0.7, random_state=37)

In [97]:
from sklearn.metrics import classification_report

### Logistic Regression

In [98]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=37).fit(X_train, y_train)
print(classification_report(y_dev, clf.predict(X_dev)))

              precision    recall  f1-score   support

           0       0.78      0.86      0.82      1317
           1       0.77      0.67      0.72       967

    accuracy                           0.78      2284
   macro avg       0.78      0.76      0.77      2284
weighted avg       0.78      0.78      0.78      2284



### Linear SVM

In [121]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC(random_state=37).fit(X_train, y_train)
print(classification_report(y_dev, lsvm.predict(X_dev)))

              precision    recall  f1-score   support

           0       0.78      0.84      0.81      1317
           1       0.76      0.68      0.72       967

    accuracy                           0.78      2284
   macro avg       0.77      0.76      0.77      2284
weighted avg       0.77      0.78      0.77      2284



### Non-Linear SVM

In [122]:
from sklearn.svm import SVC
svm = SVC(random_state=37, C=1).fit(X_train, y_train)
print(classification_report(y_dev, svm.predict(X_dev)))

              precision    recall  f1-score   support

           0       0.77      0.89      0.83      1317
           1       0.81      0.64      0.71       967

    accuracy                           0.78      2284
   macro avg       0.79      0.76      0.77      2284
weighted avg       0.79      0.78      0.78      2284



### Naive Bayes

In [101]:
import numpy as np

def calculate_probability(x, mean, std):
    exponent = np.exp(-(np.square(x-mean) / (2 * np.square(std))))
    return (1 / np.sqrt(2 * np.pi) * std) * exponent

def naive_bayes(X, y):
    miu = X.groupby(y).mean()
    sigma = X.groupby(y).std()
    probabilities = X.groupby(y).apply(lambda x: len(x)) / X.shape[0]

    predictions = np.zeros(X.shape[0])
    for i, x in enumerate(X):
        conditional_probs = {}
        for classification in np.unique(y):
            conditional_probs[classification] = probabilities.iloc[classification]
            for j, feature in enumerate(x):
                conditional_probs[classification] *=  calculate_probability(feature, miu[classification, j], sigma[classification, j])
        predictions[i] = pd.Series(conditional_probs).values.argmax()
    return predictions

In [105]:
X_train.groupby(y_train).mean()

,ablaze,accident,across,action,actually,affected,ago,air,aircraft,airplane,...,wounded,wreck,wreckage,wrong,yeah,year,yes,yet,youth,zone
is_disaster,,,,,,,,,,,,,,,,,,,,,
0,0.003636,0.005289,0.001653,0.001653,0.002645,0.000331,0.002645,0.001653,0.000000,0.001322,...,0.002314,0.006612,0.000000,0.004298,0.003636,0.014876,0.004298,0.005950,0.000661,0.004959
1,0.002170,0.021267,0.003472,0.005208,0.003472,0.009115,0.005208,0.009983,0.007812,0.009983,...,0.009115,0.004774,0.010851,0.000434,0.001736,0.022135,0.001736,0.001736,0.005642,0.002604


In [102]:
nb = naive_bayes(X_train, y_train)

KeyError: (0, 0)

In [123]:
# Test SKLearn Implementation
from sklearn.naive_bayes import GaussianNB
sk_nb = GaussianNB().fit(X_train, y_train)
print(classification_report(y_dev, sk_nb.predict(X_dev)))

              precision    recall  f1-score   support

           0       0.72      0.93      0.81      1317
           1       0.84      0.51      0.64       967

    accuracy                           0.75      2284
   macro avg       0.78      0.72      0.72      2284
weighted avg       0.77      0.75      0.74      2284

